# AS05
這裡有兩題看似要你寫Crawler，但事實上要怎麼Crawl或要怎麼Parse HTML老師都幫你寫好了。程式碼都是完整的，但卻只是某個模組功能的。所以你在這個作業要做的，實際上是看懂這些程式碼，並把這些程式碼兜起來。

這個作業要考驗你的是耐心看懂別人的程式碼，和重組別人的程式碼。

# AS05_1. Getting ibon store address
* 這題的目標是取得所有ibon生活便利站的所在地址。http://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0。
* 我已經幫你寫好，怎麼從上述網頁取得所有縣市名稱（`get()` + `BeautifulSoup`如**Get all counties***）。我也幫你寫好，怎麼取得某一個縣市的所有生活便利站（`post()` + `BeautifulSoup` 如**Get stores of a county**）。



In [211]:
import requests
from bs4 import BeautifulSoup

## Get all counties

In [212]:
url_counties = 'http://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0'
res = requests.get(url_counties)
soup = BeautifulSoup(res.text, "html.parser")
soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html>\n <head>\n  <!-- viewport -->\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta charset="utf-8"/>\n  <title id="titleNaming">\n   ibon 便利生活站 - 門市查詢-網頁版\n  </title>\n  <link href="css/pcstyle.css" rel="stylesheet" type="text/css"/>\n  <link href="css/font-awesome.min.css" rel="stylesheet"/>\n  <!-- jQuery -->\n  <script src="js/jquery-3.5.1.min.js">\n  </script>\n  <script src="js/jquery-migrate-1.4.1.min.js">\n  </script>\n  <script src="js/jquery-migrate-3.3.2.min.js">\n  </script>\n  <script src="js/selectivizr-min.js">\n  </script>\n  <script src="js/script.js">\n  </script>\n  <script src="js/commonutil.js">\n  </script>\n  <!-- 解決IE7~8看不懂html5的標籤 -->\n  <!--[if lt IE 9]>\r\n<script src=\x0b"http://html5shim.googlecode.com/svn/trunk/html5.js">\r\n</script>\r\n<![endif]-->\n  <style>\n   .subContainer {\r\n            display: none;\r\n 


- Selecting data by CSS selector https://stackoverflow.com/questions/24801548/how-to-use-css-selectors-to-retrieve-specific-links-lying-in-some-class-using-be
- Selecting node with id: https://stackoverflow.com/questions/56860183/how-to-find-allid-from-a-div-with-beautiful-soup-in-python/56860413

In [213]:
counties = []
for op in soup.select("#Class1 option"):
    print(op.text)
    counties.append(op.text)

台北市
新北市
基隆市
宜蘭縣
桃園市
新竹市
新竹縣
苗栗縣
台中市
彰化縣
南投縣
雲林縣
嘉義市
嘉義縣
台南市
高雄市
屏東縣
花蓮縣
台東縣
澎湖縣
金門縣
連江縣
南海諸島


https://stackoverflow.com/questions/50633050/scrape-tables-into-dataframe-with-beautifulsoup

## Get stores of a county

In [214]:
payloads = {'strTargetField':'COUNTY', 
            'strKeyWords':'台北市'}
url_ibon = 'https://www.ibon.com.tw/retail_inquiry_ajax.aspx'
raw = requests.post(url_ibon, data=payloads)
soup = BeautifulSoup(raw.text, "html.parser")

In [215]:
import pandas as pd
table_rows = soup.find('table').find_all('tr')
cname = ['sid', 'sname', 'addr']
rows = []
for tr in table_rows[1:]:
    tds = tr.find_all('td')
    row = []
    for td in tds:
        row.append(td.text.strip())
    rows.append(row)
pd.DataFrame(rows, columns=cname)

,sid,sname,addr
0,110817,千翔,台北市中正區許昌街17號1樓
1,111278,前港,台北市士林區後港街98之1號2號1樓
2,111636,江東,台北市中山區長安東路二段43號45-1號1樓
3,112684,塔優,台北市松山區撫遠街197號199號
4,113089,秀湖,台北市內湖區成功路四段359號1樓
...,...,...,...
861,996523,篷萊,台北市大同區寧夏路131-1號
862,996637,和業,台北市北投區中和街10號
863,996741,元大,台北市中山區遼寧街110號1樓
864,997571,聚鑫,台北市士林區延平北路六段350號


## (YOUR CODE HERE) Combine them
Output should be 
```
COUNTY	Num of Stores
台北市 	 855
新北市 	 1043
基隆市 	 80
宜蘭縣 	 108
桃園市 	 697
新竹市 	 154
新竹縣 	 183
苗栗縣 	 135
台中市 	 755
彰化縣 	 225
南投縣 	 100
雲林縣 	 120
嘉義市 	 67
嘉義縣 	 103
台南市 	 512
高雄市 	 609
屏東縣 	 163
花蓮縣 	 99
台東縣 	 60
澎湖縣 	 31
金門縣 	 25
連江縣 	 12
南海諸島 	
```

In [216]:
url_ibon = 'https://www.ibon.com.tw/retail_inquiry_ajax.aspx'
rows = []
cname = ['sid', 'sname', 'addr']

# YOUR CODE GOES FROM HERE
PayLoads = []
for county in counties:
    PayLoads.append({'strTargetField':'COUNTY', 
                     'strKeyWords':county})

ibon_count = []
for payloads in PayLoads:
    raw = requests.post(url_ibon, data=payloads)
    soup = BeautifulSoup(raw.text, "html.parser")
    table_rows = soup.find('table').find_all('tr')
    for tr in table_rows[1:]:
        tds = tr.find_all('td')
        row = []
        for td in tds:
            row.append(td.text.strip())
        rows.append(row)
    ibon_count.append(len(rows))

# count num of stores in each counties
store_n = []
for i in range(0, len(ibon_count), 1):
    if i == 0:
        difference = ibon_count[0]
    else:
        difference = ibon_count[i] - ibon_count[i-1]
    store_n.append(difference)

# bind two lists into a df
df_count = pd.DataFrame(counties, columns = ['COUNTY'])
df_count = pd.concat([df_count, pd.DataFrame(store_n, columns = ['Num of Stores'])], axis = 1)
df_count

,COUNTY,Num of Stores
0,台北市,866
1,新北市,1073
2,基隆市,83
3,宜蘭縣,112
4,桃園市,722
5,新竹市,156
6,新竹縣,191
7,苗栗縣,141
8,台中市,785
9,彰化縣,234


## AS04_1 Output Verification

### Verification 1_1
```
sid	sname	addr
0	110817	千翔	台北市中正區許昌街17號1樓
1	111278	前港	台北市士林區後港街98之1號2號1樓
2	111636	江東	台北市中山區長安東路二段43號45-1號1樓
3	112684	塔優	台北市松山區撫遠街197號199號
4	113089	秀湖	台北市內湖區成功路四段359號1樓
...	...	...	...
6132	950376	南竿	連江縣南竿鄉清水村100號
6133	952512	馬港	連江縣南竿鄉馬祖村中山路58號60號
6134	952523	北竿	連江縣北竿鄉塘岐村中山路193號195號
6135	991322	中隴	連江縣南竿鄉介壽村3鄰48-2號
6136	無符合條件的門市資料，請重新定義條件	None	None
6137 rows × 3 columns
```

In [217]:
cname = ['sid', 'sname', 'addr']
ibon_df = pd.DataFrame(rows, columns=cname)
ibon_df

,sid,sname,addr
0,110817,千翔,台北市中正區許昌街17號1樓
1,111278,前港,台北市士林區後港街98之1號2號1樓
2,111636,江東,台北市中山區長安東路二段43號45-1號1樓
3,112684,塔優,台北市松山區撫遠街197號199號
4,113089,秀湖,台北市內湖區成功路四段359號1樓
...,...,...,...
6321,950376,南竿,連江縣南竿鄉清水村100號
6322,952512,馬港,連江縣南竿鄉馬祖村中山路58號60號
6323,952523,北竿,連江縣北竿鄉塘岐村中山路193號195號
6324,991322,中隴,連江縣南竿鄉介壽村3鄰48-2號


### Verification 1_2

In [218]:
ibon_df.shape

(6326, 3)

# AS05_2 Scraping PTT comments
- 撈取Boy-Girl版最新的10頁文章（並非10則文章，是10頁的文章，約150則左右）。
- 撈取的同時，請稍微修改下面的Code以抓回Comments

## Getting 10 pages of post list
- `lastpage` 不見得是5303，撈得到就好。重點是撈10個post list頁面

In [219]:
import requests
from bs4 import BeautifulSoup

In [220]:
pre = 'https://www.ptt.cc'
links = []
lastpage = 5303
for i in range(lastpage, lastpage-10, -1):
    url = 'https://www.ptt.cc/bbs/Boy-Girl/index{}.html'.format(i)
    response = requests.get(url, timeout=(1, 2))
    soup = BeautifulSoup(response.text, 'html.parser')
    for div in soup.find_all(class_='r-ent'):
        try:
            links.append(pre + div.find(class_='title').a['href'])
        except:
            pass
    print(i, "\t", len(links))

5303 	 20
5302 	 40
5301 	 60
5300 	 80
5299 	 100
5298 	 120
5297 	 140
5296 	 160
5295 	 180
5294 	 200


## Getting comments of a specific post
- 撈到的links數也不見得會是165

In [221]:
print(links[0])
print(len(links))

https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A.48A.html
200


In [222]:
res = requests.get(links[0], timeout=(1, 2))
soup = BeautifulSoup(res.text, "html.parser")

comments = []
for push in soup.find_all(class_="push"):
    push_tag = push.find(class_='push-tag').text
    push_userid = push.find(class_='push-userid').text
    push_content = push.find(class_='push-content').text
    push_ipdatetime = push.find(class_='push-ipdatetime').text

    #print push_userid, push_content, push_ipdatetime
    comments.append({'tag': push_tag,
                     'userid':push_userid, 
                     'content':push_content, 
                     'timestamp':push_ipdatetime})
pd.DataFrame(comments)

,tag,userid,content,timestamp
0,推,mylife7126,: 推,05/03 20:52\n
1,推,jade0111,: 推,05/03 21:11\n
2,推,yichur,": 推,分手和相守,差一個字結果卻差很多",05/03 21:35\n
3,推,rouii,: 想請問原po 和男友在一起多久了呢？,05/03 21:37\n
4,推,breeze7,: 已收藏 謝謝你,05/03 22:16\n
5,推,cxz123,: 推，同4F問，請問在一起多久了？,05/03 22:53\n
6,推,rock801121,: 我跟我老婆交往三年，中間同居的時間大概不到半年,05/03 23:10\n
7,→,rock801121,: 也沒覺得感情有變淡，真的就是主動分享自己,05/03 23:11\n
8,→,rock801121,: 至於見面的時間，當然越多越好,05/03 23:14\n
9,→,rock801121,: 但如果因見面次數或時間，耽誤自己學習或工作,05/03 23:15\n


## (YOUR CODE HERE) Combine above two chunks

上面第一部分是撈10頁的posts，取得超過100個post links。
第二部分是把某一篇posts的comment用BeautifulSoup給爬梳出來，
現在你要做的事情是，把上面這兩部分結合起來，把這超過100篇的posts中的comments給抓出來。



In [223]:
from bs4 import NavigableString
all_post = []
for link in links:
    # YOUR CODE GOES FROM HERE
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.title.text
    metas = soup.find_all(class_='article-meta-value')
    # print(metas)
    author = metas[0].text
    timestamp = metas[-1].text
    
    maintext = soup.find(id = 'main-content')
    content = ""
    for text in maintext:
        if isinstance(text, NavigableString): # if text under maintext is a NavigableString:
            content += text.strip()
    
    all_post.append({'author':author, 
                     'link':link, 
                     'title':title, 
                     'timestamp':timestamp, 
                     'content':content, 
                     'comments':[]})
    
    for push in soup.find_all(class_="push"):
        push_tag = push.find(class_='push-tag').text
        push_userid = push.find(class_='push-userid').text
        push_content = push.find(class_='push-content').text
        push_ipdatetime = push.find(class_='push-ipdatetime').text

        #append push_userid, push_content, push_ipdatetime
        for post in all_post:
            post['comments'].append({'tag': push_tag, 
                                     'userid':push_userid, 
                                     'content':push_content, 
                                     'timestamp':push_ipdatetime})

## MUST Run This -> Output Verification
每則貼文的結構應該是這個樣子。
```
{
    'author': '...',
    'link': '...',
    'title': '...',
    'timestamp': '...',
    'content': '...',
    'comments': [
        {
           'tag': '推 ',
           'userid': '...',
           'content': '...',
           'timestamp': '...'
        },
        {
           'tag': '推 ',
           'userid': '...',
           'content': '...',
           'timestamp': '...'
        },
        {...},
        {...},
        {...}]
}
```
範例資料如
```
{'author': 'pauljet (拜登哥不要)',
 'link': 'https://www.ptt.cc/bbs/Boy-Girl/M.1617778263.A.0D4.html',
 'title': 'Re: [求助] 老婆家裡給房子 每次吵架就',
 'timestamp': 'Wed Apr  7 14:51:01 2021',
 'content': '你也唸一唸不會離婚啦\n\n我有這種老婆要我離婚 又不是傻了\n\n先規劃生小孩差不多\n\n房子老婆出 要小心不要被趕出去都沒你的份\n\n100萬 還是自己也在用\n\n被你講得偉大得跟什麼一樣\n\n老婆不想工作也可以唸 不是都做網拍有收入嗎？\n\n你出資金她做網拍 收入還跟你對分耶 你也有拿到盈餘\n\n你還嫌棄她不做家務 真的是 家務本來就不應該出房子的人做\n\n不知道怎麼講了 都沒要求生小孩要母姓\n\n你也腦補成入贅\n\n你這篇性別互換還不被噓爆\n\n\n--',
 'comments': [{'tag': '推 ',
   'userid': 'VVizZ',
   'content': ': 鞭屍開始XDD',
   'timestamp': ' 04/07 14:54\n'},
  {'tag': '推 ',
   'userid': 'a0913',
   'content': ': 她說又不花我錢 為什麼要催她去上班<<無業還不用花先生的錢',
   'timestamp': ' 04/07 14:54\n'},
  {'tag': '→ ',
   'userid': 'a0913',
   'content': ': 根本賺到 身在福中不知福',
   'timestamp': ' 04/07 14:55\n'},
  {'tag': '→ ',
   'userid': 'bbbing',
   'content': ': 基於安全感來個共同持有如何',
   'timestamp': ' 04/07 14:57\n'},
  {'tag': '→ ',
   'userid': 'VVizZ',
   'content': ': 當然免談 2021了還在共同持有想騙誰',
   'timestamp': ' 04/07 14:59\n'}]}
  ```

### Verification 2_1

In [225]:
all_post[-1] # all_post[0]的comments太多，改以 all_post[-1]驗證每篇貼文結構

{'author': 'susir (su sir)',
 'link': 'https://www.ptt.cc/bbs/Boy-Girl/M.1619355537.A.844.html',
 'title': '[討論] 在交友軟體發現閨蜜的男友怎麼辦？(代PO - 看板 Boy-Girl - 批踢踢實業坊',
 'timestamp': 'Sun Apr 25 20:58:55 2021',
 'content': '代PO\n\n這幾天在玩交友軟體WeDXXe的時間發現了閨蜜的男友也在玩…\n雖然沒有見過本人，但就覺得有點面熟\n後來才想起來有在閨蜜的IG上看過她跟她男友的合照\n目前推測男生應該不知道我是她的朋友\n而且我很確定他們還在一起，還沒分手！\n\n我是在想到底要不要跟朋友說「在交友軟體看到你男朋友」這件事\n還是裝作不知道？\n也不確定會不會她男友的照片是被別人盜的…\n怕說太多會被說破壞別人感情…之類的\n畢竟有時候女生對於感情的事都會比較敏感\n但才第一次聊天，就邀不認識的女生去看電影感覺就是很熟練的人，似乎很常這樣約女生的樣子\n真的為閨蜜感到很生氣！\n\n不知道各位遇到這種狀況會怎麼處理？\n裝沒這回事的話，也對閨蜜很不好…\n畢竟還是希望閨蜜找到對的人，而不是渣男\n\n--',
 'comments': [{'tag': '推 ',
   'userid': 'weltschmerz',
   'content': ': 業太美',
   'timestamp': ' 04/25 21:02\n'},
  {'tag': '噓 ',
   'userid': 'mark0204',
   'content': ': 代噓',
   'timestamp': ' 04/25 21:28\n'},
  {'tag': '噓 ',
   'userid': 'Beersheep',
   'content': ': ya',
   'timestamp': ' 04/25 21:50\n'},
  {'tag': '推 ',
   'userid': 'ggl12two2',
   'content': ': 就直接約出去 找閨蜜一起 看看是不是本人阿',
   'timestamp': ' 04/25 21:55\n'},
  {'t

### Verification 2_2

In [226]:
# MUST Run this to verify your code
# You cannot modify the following code
# should be larger than 100

print(len(all_post))
df_ptt = pd.DataFrame(all_post)
print(len(set(df_ptt.link)))

200
200


### Verification 2_3

In [227]:
# MUST Run this to verify your code
# You cannot modify the following code

unnest = df_ptt.explode('comments')
unnest

,author,link,title,timestamp,content,comments
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間 - 看板 Boy-Girl - 批踢踢實業坊,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'mylife7126', 'content..."
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間 - 看板 Boy-Girl - 批踢踢實業坊,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'jade0111', 'content':..."
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間 - 看板 Boy-Girl - 批踢踢實業坊,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'yichur', 'content': '..."
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間 - 看板 Boy-Girl - 批踢踢實業坊,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'rouii', 'content': ':..."
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間 - 看板 Boy-Girl - 批踢踢實業坊,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'breeze7', 'content': ..."
...,...,...,...,...,...,...
199,susir (su sir),https://www.ptt.cc/bbs/Boy-Girl/M.1619355537.A...,[討論] 在交友軟體發現閨蜜的男友怎麼辦？(代PO - 看板 Boy-Girl - 批踢踢實業坊,Sun Apr 25 20:58:55 2021,代PO\n\n這幾天在玩交友軟體WeDXXe的時間發現了閨蜜的男友也在玩…\n雖然沒有見過本...,"{'tag': '噓 ', 'userid': 'logLCY', 'content': '..."
199,susir (su sir),https://www.ptt.cc/bbs/Boy-Girl/M.1619355537.A...,[討論] 在交友軟體發現閨蜜的男友怎麼辦？(代PO - 看板 Boy-Girl - 批踢踢實業坊,Sun Apr 25 20:58:55 2021,代PO\n\n這幾天在玩交友軟體WeDXXe的時間發現了閨蜜的男友也在玩…\n雖然沒有見過本...,"{'tag': '噓 ', 'userid': 'Passive', 'content': ..."
199,susir (su sir),https://www.ptt.cc/bbs/Boy-Girl/M.1619355537.A...,[討論] 在交友軟體發現閨蜜的男友怎麼辦？(代PO - 看板 Boy-Girl - 批踢踢實業坊,Sun Apr 25 20:58:55 2021,代PO\n\n這幾天在玩交友軟體WeDXXe的時間發現了閨蜜的男友也在玩…\n雖然沒有見過本...,"{'tag': '→ ', 'userid': 'NeymarPtt', 'content'..."
199,susir (su sir),https://www.ptt.cc/bbs/Boy-Girl/M.1619355537.A...,[討論] 在交友軟體發現閨蜜的男友怎麼辦？(代PO - 看板 Boy-Girl - 批踢踢實業坊,Sun Apr 25 20:58:55 2021,代PO\n\n這幾天在玩交友軟體WeDXXe的時間發現了閨蜜的男友也在玩…\n雖然沒有見過本...,"{'tag': '→ ', 'userid': 'yakummi', 'content': ..."


In [228]:
# should be (1xxxxx, 6), e.g, (11393, 6)

unnest.shape

(1297872, 6)

# AS05_3 News crawler
挑選apple daily news、udn online news或ltn online news任一家，這三家線上新聞的「搜尋頁面」其搜尋結果都是用JSON傳回前端，請挑選一則新聞撰寫爬蟲，撰寫完後轉為pandas，並列印出該pandas的shape。爬蟲條件要求：

1. 每則新聞應該有標題、鏈結、內文。亦即除了爬取新聞標題、鏈結外，還需要根據鏈結來爬取內文。
2. 新聞數應不少於100則。

In [2]:
import requests
from bs4 import BeautifulSoup
links = []

# Observe request URL
    # https://news.ltn.com.tw/ajax/breakingnews/politics/2
    # https://news.ltn.com.tw/ajax/breakingnews/politics/3
    # https://news.ltn.com.tw/ajax/breakingnews/politics/4

for i in range(2, 10, 1):
    url = 'https://news.ltn.com.tw/ajax/breakingnews/politics/{}'.format(i)
    rawdata = requests.get(url).json()
    for k, v in rawdata['data'].items():
        links.append(v['url'])

In [3]:
len(links)

160

In [4]:
all_posts = []

for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    title = soup.find("h1").text
    content = ""
    for p in soup.find_all("p")[4:-6]: 
        content += p.getText().strip()
    all_posts.append({'title':title,
                     'link':link, 
                     'content':content})

In [5]:
import pandas as pd
ltn_politics = pd.DataFrame(all_posts)
ltn_politics

,title,link,content
0,政院：CPTPP11國都進口萊豬、多溝通萊豬進口會贏得支持,https://news.ltn.com.tw/news/politics/breaking...,〔記者李欣芳／台北報導〕行政院正面臨年底反萊豬公投的挑戰，由於目前多項民調顯示多數民眾反對政...
1,拜習會影響台灣情勢？ 顧立雄：台美高度互信、會前已告知,https://news.ltn.com.tw/news/politics/breaking...,〔記者謝君臨／台北報導〕拜習會之後，各方仍持續觀察相關議題。民進黨立委林楚茵今於立法院質詢時...
2,反核四！跨黨派團體50人 明赴宜蘭縣議會請願,https://news.ltn.com.tw/news/politics/breaking...,〔記者林敬倫／宜蘭報導〕民進黨、時代力量與基進黨等跨黨派團體，以及部分民間組織，近期籌組宜蘭...
3,台北通當機惹民怨 柯文哲：總是會故障,https://news.ltn.com.tw/news/politics/breaking...,〔記者楊心慧／台北報導〕台北通昨傍晚5時起陸續傳出無法使用熊好券的狀況，先是卡在讀取畫面，接...
4,民眾於投票所30公尺內刻意逗留 李進勇：這種行為就是干擾,https://news.ltn.com.tw/news/politics/breaking...,〔記者吳書緯／台北報導〕民進黨立委莊瑞雄、何欣純今在立院質詢時皆提到，台中市第二選區日前舉行...
...,...,...,...
155,高市議員籲設都更「處」 陳其邁：會盤點人力與業務,https://news.ltn.com.tw/news/politics/breaking...,〔記者王榮祥／高雄報導〕高市議員方信淵今質詢時指出，城中城大火燒出許多公安、都市更新問題，建...
156,酸顏寬恒台中七期到沙鹿「做主人」 苦苓：根本就是空飄,https://news.ltn.com.tw/news/politics/breaking...,〔即時新聞／綜合報導〕媒體人周玉蔻16日在臉書稱顏寬恒一家住在台中七期豪宅「沙鹿在地破功」，...
157,管碧玲舉數據證明 國民黨公投綁大選理由都是「騙」,https://news.ltn.com.tw/news/politics/breaking...,〔即時新聞／綜合報導〕四大公投案將於12月18日將登場，中選會近期陸續舉行公投意見發表會，民...
158,世界同志遊行主辦方改稱「台灣高雄」 外交部回應了,https://news.ltn.com.tw/news/politics/breaking...,首次上稿 00:34更新時間 06:03〔即時新聞／綜合報導〕2025世界同志遊行將在高雄舉...


In [6]:
ltn_politics.shape

(160, 3)

In [7]:
print(len(set(ltn_politics.link)))

158
